In [ ]:
import numpy as np
import pandas as pd
import os
import os.path as osp
import matplotlib.pyplot as plt
import shutil
import runpy
import scipy


In [ ]:
input_dir = input("Please enter the directory: \n")

In [ ]:
de_ba_path = 'SERS_debaseline.py'
k = 0
for dirpath, dirnames, filenames in os.walk(input_dir):
    for filenames_index in filenames :
        print(filenames_index)
        folder_name = filenames_index.split('.')[0]
        label = folder_name
        # folder_name = folder_name + '_' + str(k)  
        print(folder_name)
        print(filenames_index)
        dir = dirpath + '/' + folder_name +'/'
        print(dir)
        os.makedirs(dir, exist_ok = True)

        file = dirpath + '/' + filenames_index
        dest = dir + '/' + label + '.txt'
        de_path = dir+'/' + 'debase.py'
        shutil.move(file, dest)
        shutil.copy(de_ba_path, de_path)


In [ ]:
k = 0
for dirpath, dirnames, filenames in os.walk(input_dir):
    for filenames_index in filenames :
        if filenames_index == 'debase.py':
            #print (dirpath, dirnames, filenames_index)
            fi = dirpath + '/' + filenames_index
            file_globals = runpy.run_path(fi)


In [ ]:
wave = np.arange(400, 1550.1, 0.5, dtype=float)
feature = np.zeros((1,2301))
labels = []

k = 0
for dirpath, dirnames, filenames in os.walk(input_dir):
    # print(dirpath, dirnames, filenames)
    if dirpath[-2:] == 'sb':
        # print(filenames)
        df = pd.read_csv(dirpath + '/'+ filenames[0])
        df = df.loc[:, '400':'1550']
        df = df.to_numpy()
        feature = np.row_stack((feature, df))
        label = '_'.join(filenames[0].split("_")[1:-3])
        # print(label)
        labels.extend([label]*df.shape[0])


feature = feature[1:,:]
# label = [labels] * feature.shape[0]
print(feature.shape)
df = pd.DataFrame(feature, columns=wave)
df.insert(0, 'label', labels)
print(df)
df.to_csv(input_dir + 'RAW_SERS_dataset.csv', index= False)

In [ ]:
# Data Cleaning by 733/690 ratio >= 1.2

df = pd.read_csv(input_dir + 'RAW_SERS_dataset.csv', header = 0)

## Cleaning 
substrate_area = df.loc[:, '680.0':'700.0']
characteristic_area = df.loc[:, '720.0':'760.0']
substrate_peak = substrate_area.max(axis=1).to_numpy()
characteristic_peak = characteristic_area.max(axis=1).to_numpy()
peak_ratio = characteristic_peak / substrate_peak
peak_ratio = peak_ratio[np.logical_not(np.isnan(peak_ratio))] 


sort_ratio_index = np.argsort(peak_ratio)
print(sort_ratio_index)
ratio_threshold = 1.2  ## Retain data with ratio > 1.2
print(peak_ratio.shape)
print(peak_ratio[sort_ratio_index])
a = 0
for i in range(peak_ratio.shape[0]):
    if peak_ratio[sort_ratio_index[i]] > ratio_threshold:
        a = i
        break


qualified_ratio = sort_ratio_index[a:]
qualified_ratio = list(qualified_ratio)
qualified_ratio.sort()
df = df.iloc[qualified_ratio, :]
clean_count = df['label'].value_counts()
clean_count.to_csv(input_dir +'Cleaned_SERS_dataset_Count.csv', index= True)

df.to_csv(input_dir + 'Cleaned_SERS_dataset.csv', index= False)



In [ ]:
### For Datalabel representation 

data_path = input_dir + 'Cleaned_SERS_dataset.csv'
df = pd.read_csv(data_path, header = 0)

df['label'].replace({
# 'Adenine' :'0',
'Ecoli-BW25113_Amp0_30min': '0',   
'Ecoli-BW25113_Amp4_30min': '1', 
'Ecoli-BW25113_Amp8_30min': '1',  
'Ecoli-BW25113_Amp16_30min':'2', 
'Ecoli-BW25113_Amp32_30min':'2',   
'Ecoli-DH5alpha_Amp0_30min':'3', 
'Ecoli-DH5alpha_Amp2_30min':'4',  
'Ecoli-DH5alpha_Amp4_30min' :'4',
'Ecoli-DH5alpha_Amp8_30min'  :'5',
'Ecoli-DH5alpha_Amp16_30min'  :'5',
'Ecoli-DH5alpha-ampR_Amp0_30min':'3',
'Ecoli-DH5alpha-ampR_Amp2_30min':'6',
'Ecoli-DH5alpha-ampR_Amp4_30min':'6',
'Ecoli-DH5alpha-ampR_Amp8_30min' :'6',  
'Ecoli-DH5alpha-ampR_Amp16_30min' :'6'}, inplace=True)


df.to_csv(input_dir + 'Labeled_SERS_dataset.csv', index= False )


In [ ]:
# Train test dataset split

data_path = input_dir + 'Labeled_SERS_dataset.csv'
df = pd.read_csv(data_path, header = 0)
cols = list(df.columns)

# print(df['label'])

combine = df.to_numpy()
np.random.seed(4001)
np.random.shuffle(combine)
print (combine.shape)

percent10 = int (combine.shape[0] * 0.1)
print(percent10)

test_set = combine[0:percent10,:]
train_set = combine[percent10:,:]

test_set = pd.DataFrame(test_set, columns=cols)
train_set = pd.DataFrame(train_set, columns=cols)

test_set.to_csv(input_dir  + 'SERS_testing.csv', index= False , header = True)
train_set.to_csv(input_dir + 'SERS_training.csv', index= False , header = True)
